In [84]:
# imports

import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI


In [85]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [86]:
MODEL = 'gpt-4o-mini'
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [87]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

test_case_provider_system_prompt = "You are a test engineer who when given a scenario can provide 3 to 5 test cases. \
The test cases are in the following json format \
[[\"Step1\", \"Step2\"], [\"Step1\", \"Step2\"]] \
Each array in the json represents a test case"

In [88]:
def user_prompt_for_test_case_provider(scenario):
    user_prompt = f"provide testcases for the scenario: {scenario}"
    return user_prompt

In [89]:
def prompt_testcases_for(scenario):
    return [
        {"role": "system", "content": test_case_provider_system_prompt},
        {"role": "user", "content": user_prompt_for_test_case_provider(scenario)}
    ]

In [ ]:
prompt_testcases_for("a directed graph that will not execute cyclic nodes the second time they are encountered")

In [96]:
# And now: call the OpenAI API. You will get very familiar with this!
def generate_testcases_for(scenario):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=prompt_testcases_for(scenario),
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return result

In [ ]:
generate_testcases_for("a directed graph that will not execute cyclic nodes the second time they are encountered")

In [91]:
# Define our system prompt

test_case_consolidator_system_prompt = "Consolidate the following test cases by eliminating duplicates. When consolidating test cases: 1. Abstract specific details (e.g., names, identifiers, or labels) and focus on the core functionality or behavior being tested. 2. Treat test cases as duplicates if they test the same functionality, workflow, or structure, even if the input values or labels differ. 3. Retain test cases as unique if they test different functionalities, workflows, structures, or edge cases. Provide the output in JSON format: - `consolidatesTests`: Consolidated test cases with duplicates merged into one generalized case, while retaining unique cases. - `explanation`: A detailed explanation of why each test case was consolidated or retained as unique"

In [92]:
def user_prompt_for_test_case_consolidator(testcases):
    user_prompt = f'Consolidate the following test cases (represented as arrays of strings): {testcases}'
    return user_prompt

In [93]:
def consolidator_prompt_for_testcases(testcases):
    return [
        {"role": "system", "content": test_case_consolidator_system_prompt},
        {"role": "user", "content": user_prompt_for_test_case_consolidator(testcases)}
    ]

In [ ]:
#The arg to the function are the testcases that were generated for the scenario "a directed graph that will not execute cyclic nodes the second time they are encountered"
consolidator_prompt_for_testcases([
    ["Create directed graph with nodes A, B, C", "Add directed edges A -> B, B -> C, C -> A", "Execute graph traversal from A"],
    ["Create directed graph with nodes D, E, F", "Add directed edges D -> E, E -> F, F -> D", "Execute graph traversal from D"],
    ["Create directed graph with nodes G, H, I", "Add directed edges G -> H, H -> I, I -> H", "Execute graph traversal from G"],
    ["Create directed graph with nodes J, K, L", "Add directed edges J -> K, K -> L, L -> J", "Execute graph traversal from J"],
    ["Create directed graph with nodes M, N, O", "Add directed edges M -> N, N -> O, O -> M", "Execute graph traversal from M"]
])

In [94]:
# And now: call the OpenAI API. You will get very familiar with this!
def consolidate_testcases(testcases):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=consolidator_prompt_for_testcases(testcases),
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return result

In [ ]:
consolidate_testcases([
    ["Create directed graph with nodes A, B, C", "Add directed edges A -> B, B -> C, C -> A", "Execute graph traversal from A"],
    ["Create directed graph with nodes D, E, F", "Add directed edges D -> E, E -> F, F -> D", "Execute graph traversal from D"],
    ["Create directed graph with nodes G, H, I", "Add directed edges G -> H, H -> I, I -> H", "Execute graph traversal from G"],
    ["Create directed graph with nodes J, K, L", "Add directed edges J -> K, K -> L, L -> J", "Execute graph traversal from J"],
    ["Create directed graph with nodes M, N, O", "Add directed edges M -> N, N -> O, O -> M", "Execute graph traversal from M"]
])

In [97]:
generated_cases_patient_appointment = generate_testcases_for("Medical office patient appointment system")
print(f'generated_cases_patient_appointment: {generated_cases_patient_appointment}')
consolidated_cases_patient_appointment = consolidate_testcases(generated_cases_patient_appointment)
print(f'consolidated_cases_patient_appointment: {consolidated_cases_patient_appointment}')

generated_cases_patient_appointment: {
  "test_cases": [
    ["User logs into the system", "User navigates to the appointment booking section", "User selects a doctor", "User chooses a date and time", "User confirms the appointment"],
    ["User logs into the system", "User navigates to the appointment cancellation section", "User selects an existing appointment", "User confirms cancellation", "System displays confirmation message"],
    ["User logs into the system", "User navigates to the appointment history section", "User views past appointments", "User checks details of a specific appointment", "User logs out of the system"],
    ["User logs into the system", "User navigates to the appointment rescheduling section", "User selects an existing appointment", "User chooses a new date and time", "User confirms the rescheduling"],
    ["User logs into the system", "User attempts to book an appointment without selecting a doctor", "System prompts user to select a doctor", "User selects a 